In [ ]:
import requests
import json

In [ ]:
#### CONSTANTS
BASE_MODEL_URL = "https://app.nanonets.com/api/v2/ObjectDetection/Model/"
categories = ['object_1', 'object_2']  # 

AUTH_KEY = "<AUTH_KEY_FROM_NANONETS_APP>" ## can be foung https://app.nanonets.com/#/keys

In [ ]:
# Helper methods for creating, uploading data and training an object detection model.
def create_new_model(base_url, auth_key, categories):
    """
    function to create a new model for training
    
    Args:
    base_url: url to nanonets endpoint which will decide what type of model to create
    auth_key: authentication key provided by https://app.nanonets.com/#/keys
    categories: List of labels you want to predict/ detect
    
    return:
    model_id: a unique reference to new created model
    """
       
    payload = json.dumps({"categories" : categories})
    headers = {
        'Content-Type': "application/json",
        }

    response = requests.request(
        "POST",
        base_url,
        headers=headers,
        auth=requests.auth.HTTPBasicAuth(auth_key, ''),
        data=payload,
    )

    result = json.loads(response.text)
    print("New model created is: ", result)
    model_id, model_type, categories = (result["model_id"], result["model_type"], result["categories"])
    return model_id

def get_model_info(base_url, auth_key, model_id):
    """
    function to get/ print information about model at any time
    
    Args:
    base_url: url to nanonets endpoint which will decide what type of model to create
    auth_key: authentication key provided by https://app.nanonets.com/#/keys
    model_id: unique model_id generated at model creation time
    """
    print('%s%s'%(base_url, model_id))
    response = requests.request(
        'GET',
        '%s%s'%(base_url, model_id),
        auth=requests.auth.HTTPBasicAuth(auth_key, '')
    )
    print(response.text)
    result = json.loads(response.text)
    model_id, model_type, categories, state = (result["model_id"], result["model_type"], result["categories"], result["state"])
    return model_id, model_type, categories, state


def generate_upload_data(image_file, annotation_info, model_id):
    """
    function to translate image and annotation info into format suitable for upload
    
    Args:
    image_file[str]: full path to where the image is located
    annotation_info[str]: json formatted string of the object annotations in the image
        eg. '[{"name": "object_1", "bndbox": {"xmin": 50, "ymin": 50, "xmax": 100, "ymax": 100}}, ...]'
    model_id[str]: model id for which data needs to be uploaded
    
    Returns:
    data[Dict[str, Any]]: data that can be passed onto to the upload data method 
    """
    data = {
        'file' : open(image_file, 'rb'),
        'data' :('', '[{"filename":"%s", "object": %s}]' % (image_file.rsplit('/', 1)[1], annotation_info)),
        'modelId' :('', '%s'% model_id),
    }
    return data


def upload_data(base_url, model_id, auth_key, data):
    """
    function to upload data for a model that has been created
    
    Args:
    base_url[str]: nanonets endpoint to which the model upload request will be sent
        eg. https://app.nanonets.com/api/v2/ObjectDetection/Model/
    model_id[str]: model id of the model for which data is being uploaded generated by calling the create_model method
    auth_key[str]: authentication key provided by https://app.nanonets.com/#/keys
    data[Dict[str, Any]]: dictionary recieved from the generate_upload_data method
    """
    response = requests.post(
        '%s%s/UploadFile/'% (base_url, model_id),
        auth=requests.auth.HTTPBasicAuth(auth_key, ''),
        files=data,
    )
    print(response.text)
    
    
def train_model(base_url, auth_key, model_id):

    headers = {'authorization': 'Basic %s'%auth_key}
    querystring = {'modelId': model_id}
    response = requests.request(
        'POST',
        '%s%s/Train/'%(base_url, model_id),
        headers=headers,
        auth=requests.auth.HTTPBasicAuth(AUTH_KEY, ''),
        params=querystring,
    )
    print("training started .... ")
    print(json.loads(response.text))

# Putting it all together
# Object Detection for Millenium Falcon 

In [ ]:
import os

BASE_MODEL_URL = "https://app.nanonets.com/api/v2/ObjectDetection/Model/"
CATEGORIES = ['TieFighter', 'MillenniumFalcon']
AUTH_KEY = '<NEEDS_TO_BE_UPDATED>'

PATH_TO_JSON_ANNOTATIONS = 'annotations/json/'
PATH_TO_IMAGES = 'images/'


In [ ]:
# create_model
model_id = create_new_model(base_url=BASE_MODEL_URL, auth_key=AUTH_KEY, categories=CATEGORIES)

print("New model created: ", model_id)

In [ ]:
# generate and upload_data
# change current working directory to location where object-detection-sample-python repo is cloned
os.chdir('/Users/parvoberoi/object-detection-sample-python')
base_directory = os.getcwd()

image_directory = os.path.join(base_directory, PATH_TO_IMAGES)
annotation_directory = os.path.join(base_directory, PATH_TO_JSON_ANNOTATIONS)
annotation_files = os.listdir(annotation_directory)


# get the image and annotation info in format easy to upload
image_to_annotations = {}

for annotation_file in annotation_files:
    with open(os.path.join(annotation_directory, annotation_file)) as f:
        # check corresponding image file exists
        image_path = os.path.join(image_directory, os.path.basename(annotation_file).replace("json", "jpg"))
        if not os.path.exists(image_path):
            # skipping annotation as image does not exist
            continue
        annotation_info = f.readline().strip()
        image_to_annotations[image_path] = annotation_info


# upload the data for the model to be trained
total_images = len(list(image_to_annotations.keys()))
i = 1
for image_file, annotation_info in image_to_annotations.items():
    print("Processing Image %d / %d " % (i, total_images))
    data = generate_upload_data(image_file, annotation_info, model_id)
    upload_data(BASE_MODEL_URL, model_id, AUTH_KEY, data)
    i = i + 1

In [ ]:
# get model info 
get_model_info(BASE_MODEL_URL, AUTH_KEY, model_id)

In [ ]:
# launch training for model once all data has been uploaded
train_model(BASE_MODEL_URL, AUTH_KEY, model_id)

In [ ]:
get_model_info(BASE_MODEL_URL, AUTH_KEY, model_id)